# Lab 8: Evolutionary computation

### Consider the following example:

Determine the minimum of the function $f(x)= x_1^2+...+x_n^2$ with $x_i \in [-5.12, 5.12]$, $i \in \overline{(1, n)}$

We have an example of steady state genetic algorithm with:  representation an array of real numbers; 100 individuals; crossover $$child = \alpha \cdot (parent1 - parent2) + parent2 ;$$ mutation - reinitialise on a random position the individual's value.

In [1]:
from random import randint, random
from operator import add
from math import cos, pi


def individual(length, vmin, vmax):
    '''
    Create a member of the population - an individual

    length: the number of genes (components)
    vmin: the minimum possible value 
    vmax: the maximum possible value 
    '''
    return [(random() * (vmax - vmin) + vmin) for x in range(length)]


def population(count, length, vmin, vmax):
    """
    Create a number of individuals (i.e. a population).

    count: the number of individuals in the population
    length: the number of values per individual
    vmin: the minimum possible value 
    vmax: the maximum possible value 
    """
    return [individual(length, vmin, vmax) for x in range(count)]


def fitness(individual):
    """
    Determine the fitness of an individual. Lower is better.(min problem)
    For this problem we have the Rastrigin function
    
    individual: the individual to evaluate
    """
    n = len(individual)
    f = 0;
    for i in range(n):
        f = f + individual[i] * individual[i]
    return f


def mutate(individual, pM, vmin, vmax):
    '''
    Performs a mutation on an individual with the probability of pM.
    If the event will take place, at a random position a new value will be
    generated in the interval [vmin, vmax]

    individual:the individual to be mutated
    pM: the probability the mutation to occure
    vmin: the minimum possible value 
    vmax: the maximum possible value
    '''
    if pM > random():
        p = randint(0, len(individual) - 1)
        individual[p] = random() * (vmax - vmin) + vmin
    return individual


def crossover(parent1, parent2):
    '''
    crossover between 2 parents
    '''
    child = []
    alpha = random()
    for x in range(len(parent1)):
        child.append(alpha * (parent1[x] - parent2[x]) + parent2[x])
    return child


def iteration(pop, pM, vmin, vmax):
    '''
    an iteration

    pop: the current population
    pM: the probability the mutation to occure
    vmin: the minimum possible value 
    vmax: the maximum possible value
    '''
    i1 = randint(0, len(pop) - 1)
    i2 = randint(0, len(pop) - 1)
    if (i1 != i2):
        c = crossover(pop[i1], pop[i2])
        c = mutate(c, pM, vmin, vmax)
        f1 = fitness(pop[i1])
        f2 = fitness(pop[i2])
        '''
        the repeated evaluation of the parents can be avoided
        if  next to the values stored in the individuals we 
        keep also their fitnesses 
        '''
        fc = fitness(c)
        if (f1 > f2) and (f1 > fc):
            pop[i1] = c
        if (f2 > f1) and (f2 > fc):
            pop[i2] = c
    return pop


def main(noIteratii=10000):
    #PARAMETERS:

    #population size
    dimPopulation = 100
    #individual size
    dimIndividual = 2
    #the boundries of the search interval
    vmin = -5.12
    vmax = 5.12
    #the mutation probability
    pM = 0.01

    P = population(dimPopulation, dimIndividual, vmin, vmax)
    for i in range(noIteratii):
        P = iteration(P, pM, vmin, vmax)

    #print the best individual
    graded = [(fitness(x), x) for x in P]
    graded = sorted(graded)
    result = graded[0]
    fitnessOptim = result[0]
    individualOptim = result[1]
    print('Result: The detected minimum point after %d iterations is f(%3.2f %3.2f) = %3.2f' % \
          (noIteratii, individualOptim[0], individualOptim[1], fitnessOptim))


main()

Result: The detected minimum point after 10000 iterations is f(0.00 0.00) = 0.00


Exercise 1:  Construct a similar algorithm to the one provided as an example for the Bukin function N.6 (search the internet for this function).


In [4]:
from random import randint, random
import math


def individual():
    """
    Create a member of the population - an individual
    The range for x is typically set from -15 to -5 and for y from -3 to 3, so we set it like this
    """
    return [(random() * 10 - 15), (random() * 6 - 3)]  # x from -15 to -5, y from -3 to 3


def population(count):
    """
    Create a population of individuals.

    count: the number of individuals in the population
    """
    return [individual() for _ in range(count)]


def fitness(individual):
    """
    Calculate the Bukin function N.6 to determine the fitness of an individual.
    Lower is better.
    """
    x, y = individual[0], individual[1]
    return 100 * math.sqrt(abs(y - 0.01 * x ** 2)) + 0.01 * abs(x + 10)


def mutate(individual, pM):
    """
    Performs a mutation on an individual with the probability pM.
    """
    if random() < pM:
        # Mutate either x or y
        if randint(0, 1):
            individual[0] = random() * 10 - 15  # Mutate x
        else:
            individual[1] = random() * 6 - 3  # Mutate y
    return individual


def crossover(parent1, parent2):
    """
    Perform crossover between two parents to create a child.
    """
    child = []
    alpha = random()
    for i in range(2):
        child.append(alpha * (parent1[i] - parent2[i]) + parent2[i])
    return child


def iteration(pop, pM):
    """
    Perform one iteration of the genetic algorithm: selection, crossover, mutation.
    """
    new_pop = []
    while len(new_pop) < len(pop):
        i1, i2 = randint(0, len(pop) - 1), randint(0, len(pop) - 1)
        if i1 != i2:
            parent1, parent2 = pop[i1], pop[i2]
            child = crossover(parent1, parent2)
            child = mutate(child, pM)
            # Replace a parent with the child if the child is better
            f1, f2, fc = fitness(parent1), fitness(parent2), fitness(child)
            if f1 > fc:
                pop[i1] = child
            elif f2 > fc:
                pop[i2] = child
            new_pop.append(child)
    return pop


def main():
    # Parameters
    pop_size = 100
    num_iterations = 10000
    mutation_prob = 0.01

    # Initialize population
    pop = population(pop_size)

    # Run iterations
    for _ in range(num_iterations):
        pop = iteration(pop, mutation_prob)

    # Find the best solution
    graded = sorted([(fitness(x), x) for x in pop])
    result = graded[0]

    print('Result: The detected minimum point is f(%3.2f, %3.2f) = %3.2f' %
          (result[1][0], result[1][1], result[0]))


main()


Result: The detected minimum point is f(-10.33, 1.07) = 0.00


Consider the knapsack problem:

Consider a Knapsack with a total volum equal with $V_{max}$.

There are $n$ objects, with values $(p_i)_{n}$ and volumes $(v_i)_n$.

Solve this problem using a generationist Genetic Algorithm, with a binary representation.

Exercise 2: Initialization
Objective: Implement the initialization step of a genetic algorithm.

In [5]:
import random


def initialize_population(population_size, chromosome_length):
    """
    Generate a random population of individuals for the genetic algorithm.
    Each individual is represented as a binary chromosome of length 'chromosome_length'.
    Each bit in the chromosome can be 0 or 1, where 1 means the item is included in the knapsack.

    Args:
    population_size (int): Number of individuals in the population.
    chromosome_length (int): Number of genes in each individual's chromosome (corresponds to the number of items).

    Returns:
    list of lists: A list containing 'population_size' individuals, each a list of bits of length 'chromosome_length'.
    """
    return [[random.randint(0, 1) for _ in range(chromosome_length)] for _ in range(population_size)]

# Test the initialization step
population_size = 10
chromosome_length = 8
population = initialize_population(population_size, chromosome_length)
print(population)


[[1, 1, 1, 0, 0, 0, 1, 0], [1, 1, 1, 1, 0, 0, 0, 1], [1, 1, 1, 0, 1, 1, 0, 0], [1, 1, 0, 0, 0, 1, 1, 0], [1, 1, 1, 1, 1, 1, 1, 1], [0, 0, 1, 0, 0, 1, 0, 1], [1, 0, 0, 1, 1, 0, 0, 1], [0, 0, 1, 0, 0, 0, 0, 0], [0, 0, 1, 1, 0, 1, 0, 0], [0, 1, 1, 1, 1, 1, 0, 0]]


Exercise 3: Fitness Evaluation

Objective: Implement the fitness evaluation step of a genetic algorithm.

In [6]:
def evaluate_fitness(population):
    """
    Evaluate the fitness of each individual in the population.
    :param population: list of individuals (list of lists)
    :return: list of fitness scores
    """
    return [sum(chromosome) for chromosome in population]


# Test the fitness evaluation step
fitness_scores = evaluate_fitness(population)
print(fitness_scores)


[4, 5, 5, 4, 8, 3, 4, 1, 3, 5]


Exercise 4: Selection

Objective: Implement the selection step of a genetic algorithm.

In [7]:
def select_parents(population, fitness_scores):
    # select two parents from the population based on the fitness -
    # the better the fitness, the higher the chance to be selected
    return random.choices(population, weights=fitness_scores, k=2)



# Test the selection step
parents = select_parents(population, fitness_scores)
print(parents)


[[1, 1, 1, 0, 0, 0, 1, 0], [1, 1, 1, 1, 0, 0, 0, 1]]


Exercise 5: Crossover

Objective: Implement the crossover step of a genetic algorithm.

In [55]:
def crossover(parents):
    # create two new offspring by combining the parents
    # IN:  parents
    # OUT: offspring
    return [parents[0][:len(parents[0]) // 2] + parents[1][len(parents[1]) // 2:],
            parents[1][:len(parents[1]) // 2] + parents[0][len(parents[0]) // 2:]]


# your code here

# Test the crossover step
offspring = crossover(parents)
print(offspring)


[[1, 1, 1, 0, 0, 0, 0, 1], [1, 1, 1, 1, 0, 0, 1, 0]]


Exercise 6: Mutation

Objective: Implement the mutation step of a genetic algorithm.

In [22]:
def mutate(chromosome, mutation_rate):
    # mutate the chromosome by randomly flipping bits
    # IN:  chromosome, mutation_rate
    # OUT: mutated_chromosome
    return [1 - gene if random.random() < mutation_rate else gene for gene in chromosome]

# your code here

# Test the mutation step
mutation_rate = 0.1
mutated_offspring = [mutate(child, mutation_rate) for child in offspring]
print(mutated_offspring)


[[0, 1, 1, 1, 0, 0, 0, 1], [1, 1, 1, 1, 0, 0, 1, 0]]


Exercise 7: Complete Genetic Algorithm

Objective: Combine all the steps of a genetic algorithm to solve a specific problem.

In [25]:
def genetic_algorithm(population_size, chromosome_length, generations, mutation_rate):
    # complete genetic algorithm
    # IN:  population_size, chromosome_length, generations, mutation_rate
    # OUT: population

    population = initialize_population(population_size, chromosome_length)

    for _ in range(generations):
        # Fitness evaluation
        fitness_scores = evaluate_fitness(population)

        # Selection
        new_population = []
        while len(new_population) < population_size:
            parents = select_parents(population, fitness_scores)

            # Crossover
            offspring = crossover(parents)

            # Mutation
            offspring = [mutate(child, mutation_rate) for child in offspring]

            # Add new offspring to the new generation
            new_population.extend(offspring[:2])  # Ensure the population size remains constant

        # Replace the population with the new generation
        population = new_population[:population_size]  # Ensure population does not exceed initial size

    return population


# Test the complete genetic algorithm
population_size = 10
chromosome_length = 8
generations = 100
mutation_rate = 0.1

final_population = genetic_algorithm(population_size, chromosome_length, generations, mutation_rate)
print(final_population)


[[1, 1, 1, 1, 1, 0, 1, 0], [1, 0, 0, 1, 0, 0, 1, 0], [1, 0, 1, 0, 0, 0, 1, 1], [1, 1, 1, 1, 1, 1, 0, 1], [0, 1, 1, 1, 1, 1, 1, 0], [1, 0, 1, 1, 1, 0, 0, 1], [1, 1, 1, 1, 1, 1, 0, 0], [1, 0, 1, 1, 1, 0, 1, 0], [1, 0, 0, 1, 1, 1, 1, 0], [1, 1, 0, 1, 1, 1, 1, 0]]


Exercise 8: Extract the result from the final population

Objective: Get the best individual from the final population.


In [41]:
# determine the best individual from the final population and print it out

# your code here
fitness_scores = evaluate_fitness(final_population)
best_individual = final_population[fitness_scores.index(max(fitness_scores))]
best_individual


[1, 1, 1, 1, 1, 1, 0, 1]